<a href="https://colab.research.google.com/github/Ahmed-Hossam-Aldeen/Slowed-Reverb/blob/main/Slow_and_Reverb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install youtube_dl
!pip install pydub
!pip install simple_youtube_api
!pip install pysndfx
!pip install glitch_this
!pip install colorthief
!pip install colormap
!pip install easydev

!apt -qq install -y sox

In [ ]:
!pip install --upgrade opencv-python
!pip install --upgrade opencv-contrib-python

In [3]:
#SLOWED---WITH---LINK
from __future__ import unicode_literals
import youtube_dl
import os
from scipy.io import wavfile
import wave
import os
import numpy as np
import matplotlib.pyplot as plt
from pydub import AudioSegment
import logging
import librosa as lr
import soundfile as sf
from pysndfx.dsp import AudioEffectsChain

link = input("Enter Link: ")
speed = input("Enter Speed: ")
choice = int(input("1.Enter color / 2.Dominants color from thumbnail "))

if choice == 1:
   color = input("Enter color name:")

upload = "y"
color = "orange"
ydl_opts = {
    #'outtmpl': '/' + str(input) +'.%(mp3)s',
    'format': 'bestaudio/best',
    'postprocessors': [{
        'key': 'FFmpegExtractAudio',
        'preferredcodec': 'mp3',
        'preferredquality': '320',

    }],
}

with youtube_dl.YoutubeDL(ydl_opts) as ydl:
    info_dict = ydl.extract_info(link, download=False)
    title = info_dict.get('title', None)
    id=video_id = info_dict.get("id", None)
    ful= title+' '+"[slowed + reverb]"+'.'+'mp3'
    con= title+'-'+id+'.'+'mp3'
    conc=con.replace("|","_").replace("/","_")
    full=ful.replace("|","_").replace("/","_")
    print(conc)
    print(full)
    inputt=title+'.'+'mp3'
    inputt= inputt.replace("|","_").replace("/","_")
    ydl.download([link])
os.rename('./%s'%conc, "input.mp3")

#------------------------------------------------------------------------------#
 #Define conversion format function
def trans_mp3_to_wav(filepath):
    song = AudioSegment.from_mp3(filepath)
    song.export("temp.wav", format="wav")
trans_mp3_to_wav("./input.mp3")


logger = logging.getLogger('pysndfx')
logger.setLevel(logging.DEBUG)
apply_audio_effects =     (AudioEffectsChain()
     .speed(speed)
     .reverb()
    )
infile = './temp.wav'
mono, sr = lr.load(infile, sr=None)
stereo, _ = lr.load(infile, sr=None, mono=False)
outfile = 'output.wav'

def test_file_to_file():
    apply_audio_effects(infile, outfile)
    y = lr.load(outfile, sr=None, mono=False)[0]
    sf.write('output.wav', y.T, sr)
    assert lr.util.valid_audio(y, mono=False)
test_file_to_file()
os.remove('./temp.wav')

import pydub
sound = pydub.AudioSegment.from_wav("./output.wav")
sound.export("final.mp3", format="mp3")
os.remove('./output.wav')

os.rename("final.mp3","%s"%full)
os.rename("input.mp3","%s"%inputt)
##########################--Determine color of video and thumbnail---######################################
if upload == "y":
  from simple_youtube_api.Channel import Channel
  from simple_youtube_api.LocalVideo import LocalVideo
  import os
  import requests # to get image from the web
  import shutil # to save it locally

  name = "%s" %full
  name = name.replace(".mp3","")
  if len(name)>70:
    name=name[0:42]+" [slowed + reverb]"
  print(name)
  os.system(f"youtube-dl --write-thumbnail --skip-download --output img {link}")
  if os.path.isfile('img.png'):
    os.rename("img.png","img.jpg")
  elif os.path.isfile('img.webp'):
    os.rename("img.webp","img.jpg")

  import PIL
  from PIL import Image 
  img = PIL.Image.open("img.jpg")
  width, height = img.size
  if(width<1000 and height <600):
  ###################################################################################################
    ## Set up the image URL and filename
    image_url = input ("if Low quality Found, Enter new image link :") 
    filename = 'img.jpg'
    # Open the url image, set stream to True, this will return the stream content.
    r = requests.get(image_url, stream = True)
    # Check if the image was retrieved successfully
    if r.status_code == 200:
        # Set decode_content value to True, otherwise the downloaded image file's size will be zero.
        r.raw.decode_content = True

        # Open a local file with wb ( write binary ) permission.
        with open(filename,'wb') as f:
            shutil.copyfileobj(r.raw, f)

        print('Image sucessfully Downloaded: ',filename)
    else:
        print('Image Couldn\'t be retreived')

  
 
  if choice != 1:
      from colorthief import ColorThief
      from colormap import rgb2hex
      color_thief = ColorThief('img.jpg')
      # get the dominant color
      palette = color_thief.get_palette(color_count=4)
      print(palette)
      newPalette= palette[2]
      print(newPalette)
      color = rgb2hex(newPalette[0],newPalette[1],newPalette[2])
      import seaborn as sns
      domColor = np.array(newPalette) / 255.0
      sns.palplot(domColor)      
  #########################################################################################################
  !glitch_this img.jpg 2.2 -s -c -f
  ###############################################-Video-#############################################################
  full2 = "'"+full+"'"
  command = (f'ffmpeg -i {full2} -i glitched_img.jpg -filter_complex "[0:a]aformat=channel_layouts=mono,showwaves=s=1280x202:mode=line:colors={color}|black[sw];[1][sw]overlay=(W-w)/2:(H-h)/2:format=auto,format=yuv420p[v]" -map "[v]" -map 0:a -movflags +faststart videoMono.mp4')
  os.system(command)

  # loggin into the channel
  channel = Channel()
  channel.login("./drive/MyDrive/0Slowed files/client_secret.json", "./drive/MyDrive/0Slowed files/credentials.storage")

  # setting up the video that is going to be uploaded
  video = LocalVideo(file_path="videoMono.mp4")

  # setting snippet
  video.set_title(name)
  video.set_description("I don't own music in this video, Uploaded using youtube API and Google Collab")
  video.set_tags(["this", "tag"])
  video.set_category("music")
  video.set_default_language("en-US")

  # setting status
  video.set_embeddable(True)
  video.set_license("creativeCommon")
  video.set_privacy_status("public")
  video.set_public_stats_viewable(True)

  # setting thumbnail
  video.set_thumbnail_path('glitched_img.jpg')

  # uploading video and printing the results
  video = channel.upload_video(video)
  print(video.id)
  print(video)

  # liking video
  video.like()

  os.remove('./videoMono.mp4')

Enter Link: https://www.youtube.com/watch?v=4ZqGjsfw--w
Enter Speed: 0.8
1.Enter color / 2.Dominants color from thumbnail 1
Enter color name:pink
[youtube] 4ZqGjsfw--w: Downloading webpage
[youtube] 4ZqGjsfw--w: Downloading player 450209b9
Abo El Anwar X Lil Baba - MSH HA7EL -  ليل بابا  مش هحل  X أبو الأنوار (Official Music Video)-4ZqGjsfw--w.mp3
Abo El Anwar X Lil Baba - MSH HA7EL -  ليل بابا  مش هحل  X أبو الأنوار (Official Music Video) [slowed + reverb].mp3
[youtube] 4ZqGjsfw--w: Downloading webpage
[download] Destination: Abo El Anwar X Lil Baba - MSH HA7EL -  ليل بابا  مش هحل  X أبو الأنوار (Official Music Video)-4ZqGjsfw--w.webm
[download] 100% of 5.06MiB in 01:20
[ffmpeg] Destination: Abo El Anwar X Lil Baba - MSH HA7EL -  ليل بابا  مش هحل  X أبو الأنوار (Official Music Video)-4ZqGjsfw--w.mp3
Deleting original file Abo El Anwar X Lil Baba - MSH HA7EL -  ليل بابا  مش هحل  X أبو الأنوار (Official Music Video)-4ZqGjsfw--w.webm (pass -k to keep)
Abo El Anwar X Lil Baba - MSH HA7EL 

Upload:   0% |/                                      | ETA:  --:--:--   0.0 s/B

Video size: 52694862 bytes


Upload: 100% ||                                      | Time:  0:00:21  47.1 B/s


kixl9Z2_byU
Title: 
Description: 
 Tags:[]


In [ ]:
from simple_youtube_api.Channel import Channel
from simple_youtube_api.LocalVideo import LocalVideo
import os
import requests # to get image from the web
import shutil # to save it locally

name = "%s" %full
print(name)
os.system(f"youtube-dl --write-thumbnail --skip-download --output img {link}")
if os.path.isfile('img.png'):
 os.rename("img.png","img.jpg")
elif os.path.isfile('img.webp'):
 os.rename("img.webp","img.jpg")

###################################################################################################
# ## Set up the image URL and filename
# image_url = "https://i.pinimg.com/originals/94/cb/45/94cb454c16bcc05fba36e0e7444488a1.png"
# filename = 'img.jpg'
# # Open the url image, set stream to True, this will return the stream content.
# r = requests.get(image_url, stream = True)
# # Check if the image was retrieved successfully
# if r.status_code == 200:
#     # Set decode_content value to True, otherwise the downloaded image file's size will be zero.
#     r.raw.decode_content = True

#     # Open a local file with wb ( write binary ) permission.
#     with open(filename,'wb') as f:
#         shutil.copyfileobj(r.raw, f)

#     print('Image sucessfully Downloaded: ',filename)
# else:
#     print('Image Couldn\'t be retreived')
#########################################################################################################
!glitch_this img.jpg 2.2 -s -c -f

full2 = "'"+full+"'"
command = (f'ffmpeg -i {full2} -i img.jpg -filter_complex "[0:a]aformat=channel_layouts=mono,showwaves=s=1280x202:mode=line:colors=red|black[sw];[1][sw]overlay=(W-w)/2:(H-h)/2:format=auto,format=yuv420p[v]" -map "[v]" -map 0:a -movflags +faststart videoMono.mp4')
os.system(command)

# loggin into the channel
channel = Channel()
channel.login("./drive/MyDrive/0Slowed files/client_secret.json", "./drive/MyDrive/0Slowed files/credentials.storage")

# setting up the video that is going to be uploaded
video = LocalVideo(file_path="videoMono.mp4")

# setting snippet
video.set_title(name)
video.set_description("I don't own music in this video, Uploaded using youtube API and Google Collab")
video.set_tags(["this", "tag"])
video.set_category("music")
video.set_default_language("en-US")

# setting status
video.set_embeddable(True)
video.set_license("creativeCommon")
video.set_privacy_status("public")
video.set_public_stats_viewable(True)

# setting thumbnail
video.set_thumbnail_path('glitched_img.jpg')

# uploading video and printing the results
video = channel.upload_video(video)
print(video.id)
print(video)

# liking video
video.like()

os.remove('./videoMono.mp4')

In [ ]:
import os

print(full)
color = '#792F34'
full2 = "'"+full+"'"
command = (f'ffmpeg -i {full2} -i img.jpg -filter_complex "[0:a]aformat=channel_layouts=mono,showwaves=s=1280x202:mode=line:colors={color}|black[sw];[1][sw]overlay=(W-w)/2:(H-h)/2:format=auto,format=yuv420p[v]" -map "[v]" -map 0:a -movflags +faststart videoMono.mp4')
os.system(command)
print(command)

In [ ]:
# importing the module
import PIL
from PIL import Image 
img = PIL.Image.open("img.jpg")
width, height = img.size
if(width<1200 and height <720):
  print(img.size)
    


## SLOWED with LINK and GIF ✨



In [ ]:
from __future__ import unicode_literals
import youtube_dl
import os
from scipy.io import wavfile
import wave
import os
import numpy as np
import matplotlib.pyplot as plt
from pydub import AudioSegment
import logging
import librosa as lr
import soundfile as sf
from pysndfx.dsp import AudioEffectsChain
import requests # to get image from the web
import shutil # to save it locally
import PIL
from PIL import Image 

link = input("Enter Link: ")
speed = input("Enter Speed: ")
gif_url = input ("Enter gif link :") 
upload = "y"

ydl_opts = {
    #'outtmpl': '/' + str(input) +'.%(mp3)s',
    'format': 'bestaudio/best',
    'postprocessors': [{
        'key': 'FFmpegExtractAudio',
        'preferredcodec': 'mp3',
        'preferredquality': '320',
    }],
}

with youtube_dl.YoutubeDL(ydl_opts) as ydl:
    info_dict = ydl.extract_info(link, download=False)
    title = info_dict.get('title', None)
    id=video_id = info_dict.get("id", None)
    ful= title+' '+"[slowed + reverb]"+'.'+'mp3'
    con= title+'-'+id+'.'+'mp3'
    conc=con.replace("|","_").replace("/","_")
    full=ful.replace("|","_").replace("/","_")
    print(conc)
    print(full)
    inputt=title+'.'+'mp3'
    inputt= inputt.replace("|","_").replace("/","_")
    ydl.download([link])
os.rename('./%s'%conc, "input.mp3")

#Define conversion format function
def trans_mp3_to_wav(filepath):
    song = AudioSegment.from_mp3(filepath)
    song.export("temp.wav", format="wav")

def test_file_to_file():
    apply_audio_effects(infile, outfile)
    y = lr.load(outfile, sr=None, mono=False)[0]
    sf.write('output.wav', y.T, sr)
    assert lr.util.valid_audio(y, mono=False)    
#------------------------------------------------------------------------------#
trans_mp3_to_wav("./input.mp3")
logger = logging.getLogger('pysndfx')
logger.setLevel(logging.DEBUG)
apply_audio_effects =     (AudioEffectsChain()
     .speed(speed)
     .reverb()
    )
infile = './temp.wav'
mono, sr = lr.load(infile, sr=None)
stereo, _ = lr.load(infile, sr=None, mono=False)
outfile = 'output.wav'
test_file_to_file()
os.remove('./temp.wav')

import pydub
sound = pydub.AudioSegment.from_wav("./output.wav")
sound.export("final.mp3", format="mp3")
os.remove('./output.wav')

os.rename("final.mp3","%s"%full)
os.rename("input.mp3","%s"%inputt)

###########################################################################################################
## Set up the GIF URL and filename
filename = 'gif.gif'
# Open the url image, set stream to True, this will return the stream content.
r = requests.get(gif_url, stream = True)
# Check if the image was retrieved successfully
if r.status_code == 200:
    # Set decode_content value to True, otherwise the downloaded image file's size will be zero.
    r.raw.decode_content = True
    # Open a local file with wb ( write binary ) permission.
    with open(filename,'wb') as f:
        shutil.copyfileobj(r.raw, f)
    print('Image sucessfully Downloaded: ',filename)
else:
    print('Image Couldn\'t be retreived')
#################################################################################################
gif = PIL.Image.open("gif.gif")
width, height = gif.size

if(width<400 and height <400):
    print("ah shit here we go again")   
    import os
    from os import path
    #check if dir exists
    if path.exists("gif images") == False:
        os.mkdir("./gif images")
        
    #get num of frames
    with Image.open('gif.gif') as im:
        num_key_frames = im.n_frames

    #convert to png
    with Image.open('gif.gif') as im:
        for i in range(num_key_frames):
            im.seek(im.n_frames // num_key_frames * i)
            im.save('./gif images/{}.png'.format(i))

    import cv2
    from cv2 import dnn_superres
    import os
    from os import path
    import imageio
    import re
    images = []

    def sorted_alphanumeric(data):
        convert = lambda text: int(text) if text.isdigit() else text.lower()
        alphanum_key = lambda key: [ convert(c) for c in re.split('([0-9]+)', key) ] 
        return sorted(data, key=alphanum_key)

    filenames = sorted(os.listdir("./gif images/"))
    filenames = sorted_alphanumeric(filenames)
    #print(filenames)      
    for filename in filenames:
        images.append((f'./gif images/{filename}'))
    #print(images)   

    # Create an SR object
    sr = dnn_superres.DnnSuperResImpl_create()
    # Set the desired model and scale to get correct pre- and post-processing
    path = "./drive/MyDrive/0Slowed files/models/ESPCN_x3.pb"
    sr.setModel("espcn", 3)
    # Read image
    for filename in images:
        image = cv2.imread(filename)
        sr.readModel(path)
        # Upscale the image
        result = sr.upsample(image)
        # Save the image
        cv2.imwrite(f'./{filename}', result)
        print("done")

    imagez= []
    for filename in filenames:
        imagez.append(imageio.imread(f'./gif images/{filename}'))
    imageio.mimsave('./gif images/movie.gif', imagez)

    shutil.move('./gif images/movie.gif', "./")
    shutil.rmtree("./gif images/")     
##########################----------make video from Gif---######################################
if upload == "y":
  from simple_youtube_api.Channel import Channel
  from simple_youtube_api.LocalVideo import LocalVideo
  import os
  import requests # to get image from the web
  import shutil # to save it locally

  name = "%s" %full
  name = name.replace(".mp3","")
  if len(name)>70:
    name=name[0:42]+" [slowed + reverb]"
  print(name)

  ## Set up the GIF URL and filename
  gifname = 'gif.gif'
  if(width<400 and height <400):
    gifname = 'movie.gif'


  # load audio file
  import moviepy.editor as mp
  import math
  output_path = "./slowed-reverb.mp4"
  audio_clip = mp.AudioFileClip(full)
  # load video
  video_clip = mp.VideoFileClip(gifname)

  # loop the gif for the duration of the audio
  num_loops = math.ceil(audio_clip.duration / video_clip.duration)
  video_clip2 = video_clip.loop(n=num_loops)
  # add the audio to the video
  video_clip3 = video_clip2.set_audio(audio_clip)
  # save result
  video_clip3.write_videofile(output_path, verbose=False)


  # loggin into the channel
  channel = Channel()
  channel.login("./drive/MyDrive/0Slowed files/client_secret.json", "./drive/MyDrive/0Slowed files/credentials.storage")
  # setting up the video that is going to be uploaded
  video = LocalVideo(file_path="slowed-reverb.mp4")

  # setting snippet
  video.set_title(name)
  video.set_description("I don't own music in this video, Uploaded using youtube API and Google Collab")
  video.set_tags(["this", "tag"])
  video.set_category("music")
  video.set_default_language("en-US")
  # setting status
  video.set_embeddable(True)
  video.set_license("creativeCommon")
  video.set_privacy_status("public")
  video.set_public_stats_viewable(True)
  # setting thumbnail
  #video.set_thumbnail_path('glitched_img.jpg')
  # uploading video and printing the results
  video = channel.upload_video(video)
  print(video.id)
  print(video)
  # liking video
  video.like()

os.remove('./slowed-reverb.mp4')



Enter Link: https://www.youtube.com/watch?v=OHZd1KoLcPk
Enter Speed: 0.8
Enter gif link :https://i.giphy.com/media/k5zu35npVsYfgZQwwl/200.gif
[youtube] OHZd1KoLcPk: Downloading webpage
Betadeen-OHZd1KoLcPk.mp3
Betadeen [slowed + reverb].mp3
[youtube] OHZd1KoLcPk: Downloading webpage
[download] Destination: Betadeen-OHZd1KoLcPk.webm
[download] 100% of 3.50MiB in 00:43
[ffmpeg] Destination: Betadeen-OHZd1KoLcPk.mp3
Deleting original file Betadeen-OHZd1KoLcPk.webm (pass -k to keep)
Image sucessfully Downloaded:  gif.gif
ah shit here we go again
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
Betadeen [slowed + reverb]


100%|█████████▉| 2380/2381 [03:49<00:00, 10.37it/s]
Upload:   0% |/                                      | ETA:  --:--:--   0.0 s/B

Video size: 43550893 bytes


Upload: 100% ||                                      | Time:  0:00:05 173.2 B/s


3VWGdbsOXRI
Title: 
Description: 
 Tags:[]


In [ ]:
shutil.rmtree("./gif images/")  

In [ ]:
os.remove("gif.gif")
os.remove("movie.gif")